## pip

In [ ]:
!pip install pororo
!pip install transformers
!pip install jupyter-tensorboard
!pip install sentence_transformers

In [ ]:
pip install easydict

## 라이브러리

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import transformers
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration
import re
from kobert_tokenizer import KoBERTTokenizer

In [ ]:

## Version.
print("[VERSION]")
print(f"torch: {torch.__version__}")
print(f"transformers: {transformers.__version__}")
     

## 데이터 로드

In [ ]:
df= pd.read_csv('/home/kic/kpmg/base_report.csv',encoding='utf-8',error_bad_lines=False,engine='python')
df

In [4]:
text =df['full_text']

## 추가데이터

In [5]:
#추가 데이터 학습
def __init__(self, hparams, **kwargs):
        super(KoBARTConditionalGeneration, self).__init__(hparams, **kwargs)
        self.model = BartForConditionalGeneration.from_pretrained(get_pytorch_kobart_model(df.iloc['Busines_report'][:500]))
        self.model.train()
        self.bos_token = '<s>'
        self.eos_token = '</s>'
        self.pad_token_id = 0
        self.tokenizer = get_kobart_tokenizer(df.iloc[:300,['full_text']])


## 문장구분

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')
#tokenizer = KoBERTTokenizer.from_pretrained(df.loc[:300])

In [7]:
#요약
def summarize(text):
    
    context_tokens = ['<s>'] + tokenizer.tokenize(text) + ['</s>']
    input_ids = tokenizer.convert_tokens_to_ids(context_tokens) 

    if len(input_ids) < 512:   
            while len(input_ids) < 512: 
                input_ids += [tokenizer.pad_token_id] 

    else:
        input_ids = input_ids[:512 - 1] + [   
            tokenizer.eos_token_id]

    res_ids = model.generate(torch.tensor([input_ids]),
                                        max_length=1024,
                                        num_beams=3,
                                        no_repeat_ngram_size = 3,
                                        eos_token_id=tokenizer.eos_token_id,
                                        bad_words_ids=[[tokenizer.unk_token_id]])        
    
    res = tokenizer.batch_decode(res_ids.tolist(), skip_special_tokens=True)[0]
    
    return res

In [8]:
# 문장 구분
def sent_spllit(text):
    k = text.split('\n')
    for sent in range(len(k) - 1):
        k[sent] = k[sent] + " "
        if k[sent][0] == " ":
            k[sent] = k[sent][1:]
    return k

In [9]:
#접두사 요소 생성
def starts_with(text):
    list =[]

    for i in range(100):
        list.append(f'{i}.')

    list =tuple(list)

    #항목별('1.','2.'인덱스 추출)
    k= sent_spllit(text)
    indexs =[]
    for i in range(len(k)):
        if k[i].startswith(list):
            indexs.append(i)

    lists =[]
    i = 0 
    for i in (range(len(indexs))):
        if i >=(len(indexs)-1):
            a = k[indexs[i-1]:]
            a ='\n'.join(a)
            lists.append(a)

        else:
            a= k[indexs[i]:indexs[i+1]]
            a='\n'.join(a)
            lists.append(a)

        i +=1
    return lists

In [10]:
#요소별 summarize
def sum_list(list):
    sum=[]
    for i in tqdm(range(len(list))):
        sum.append(summarize(list[i]))
    return sum

In [11]:
len(df['full_text'][500:])

3407

In [ ]:
results.clear()

In [33]:
result.clear()

In [ ]:
results =[]
i=500
for i in tqdm(df['full_text'][500:]):
    list = starts_with(i)
    result  = sum_list(list)
    results.append(result)

In [21]:
results[0]

['2021년 01월 01일부터 2021년 12월 31일까지 (주)대성미생물연구소  202(당) 기    5. 재무제표 주석  2021년 02월 011일부터 2020년 12월 31일 까지  2021월 021일부터  2018년 12월 31일까지의  2022년 재무제표를   (주)당(주)대성 미생물연구소가   지정했다. ',
 "2000년 3월 31일자로 한국증권업협회 및 코스닥증권시장에 등록된 '대성미생물연구소'는 1966년 2월 4일 설립되어 동물의약품류의 제조 및 판매업을 영위하고 있으며  사업의 주요 주주이며 주요 주주현황은 다음과 같다.",
 '2021년 1월 1일로 개시하는 회계기간부터 한국채택국제회계기준을 적용하고 있는 바른미래당사는 회계정책과 공시의 변경폭이 크지만 재무제표 작성시 중요한 회계추정의 사용을 허용하고 있으며, 회계정책을 적용함에 있어 경영진의 판단을 요구하고 있다.',
 ' 2018당사는 보유현금과 요구불예금, 유동성이 매우 높고 확정된 금액의 현금으로 전환이 용이하고 가치변동의 위험이 경미한 단기 투자자산을 현금 및 현금성자산으로 분류하고 있으며 최초 인식 후에 원가에서 감가상각누계액과 손상차손누계액을 차감한 금액을 장부금액으로 표시하고 있다.',
 '당기말 및 전기말 현재 매출채권의 약정회수기일기준 연령분석은 다음과 같으며 당기말과 전기말 매출채권은 장부금액과 공정가치와의 차이가 중요하지 않아 상각후원가로 측정하였습니다.',
 '당사는 진부화된 재고를 폐기처분하므로 보유하고 있지 않으며 재고자산 3위 3위인 당기말과 전기말 현재 재고자산의 내역은 다음과 같다.',
 '6. 관계기업투자 등 출자지분에 대한 내역은 다음과 같다.',
 '당사가 토지, 건물 및 기계장치에 대하여 채권최고액 28,264백만원의 근저당권이 설정되어 있고 당기말 현재 당사가 보유하고 있는 토지의 공시지가 내역은 다음과 같다.',
 ' 2018당사는 상기보험외에 종업원에 대한 산재보험과 차량운반구에 대한 종합보험 및  러시아의 보험에 가입하고 있으며  2018~

## result 데이터프레임 만들기

In [ ]:

df2 = pd.DataFrame()
df2 =df2.append(pd.Series(results), ignore_index = True)

In [31]:
df2=df2.transpose()
df2

,0
0,[2021년 01월 01일부터 2021년 12월 31일까지 (주)대성미생물연구소 ...
1,[5. 재무제표 주석 알렉산드로이드제44기 2021년 12월 31일 현재 러시아의...
2,[5. 재무제표 주석 \n주석 \n제 21(당) 기 : 2021년 12월 31일 현...
3,"[재무제표 주석, 1997년 2월 14일 설립되어 연간 120만톤의 조강 생산능력을..."
4,[5. 재무제표 주석 5 기 : 2021년 1월 1일 부터 2021년 12월 31일...
...,...
435,"[재무제표 주석, (주)에너지토크는 자동조정 및 제어 장비 제조판매업을 주요사업으로..."
436,"[재무제표 주석, 1963년 3월 9일에 설립되어 플랜트 엔지니어링 용역, 종합건설..."
437,"[5. 재무제표 주석 5., 외환신용카드 주식회사를 흡수합병하고 법인명을 하나은행..."
438,"[2021년 12월 31일 현재 \n주식회사 인디에프, 당사의 의류 제조 및 판매를..."


In [32]:
df2 = df2.to_csv('/home/kic/kpmg/summary[:439].csv')


In [ ]:
df3 = pd.read_csv('/home/kic/kpmg/summary[:439].csv',encoding='utf-8')
df3

In [ ]:
df1 = pd.read_csv('/home/kic/kpmg/summary[:439].csv')
df1

In [ ]:
df = pd.read_csv('/home/kic/kpmg/summary[:499].csv')
df

In [ ]:
df4 = pd.concat([df, df1])
df4.rename(columns={'0':'summary'},inplace=True)
df4

In [ ]:
df2 = pd.DataFrame(index=range(0,2967), columns=['summary'])
df2

In [ ]:
df4 = pd.concat([df4, df2])
df4

In [ ]:
df4.drop(['Unnamed: 0'],axis=1)

In [75]:
df5 = df4.to_csv('/home/kic/kpmg/summary.csv',encoding='utf-8')